In [1]:
import numpy as np
import pandas as pd
#from google.colab import drive
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import OneClassSVM

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

from joblib import dump, load

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import LabelEncoder

import zipfile

In [2]:
sns.set_theme()
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Carregando os dados

In [14]:
df_benign = pd.read_csv(("data/dados benignos/logs tratados/Treated_Benign_Traffic.csv"))
df_benign

,Rel Time,DLC,Arb ID,Byte 0,Byte 1,Byte 2,Byte 3
0,0.000000,1,165,100,0,0,0
1,0.001296,1,165,100,0,0,0
2,0.001552,1,165,100,0,0,0
3,0.000112,1,165,100,0,0,0
4,0.000906,1,165,100,0,0,0
...,...,...,...,...,...,...,...
393885,0.000184,2,194,54,0,0,0
393886,0.046381,4,192,244,146,0,37
393887,0.076806,3,193,0,0,0,0
393888,0.000133,2,194,54,0,0,0


In [15]:
df_malicious = pd.read_csv("data/ataques/dados_aleatorios_1.csv")
df_malicious

,Rel Time,DLC,Arb ID,Byte 0,Byte 1,Byte 2,Byte 3
0,0.205494,3,74,146,15,251,125
1,0.124802,2,222,10,43,197,147
2,0.047389,1,70,79,220,195,180
3,0.361882,3,156,26,113,192,130
4,0.450875,4,129,209,249,55,67
...,...,...,...,...,...,...,...
49995,0.508828,2,193,104,139,17,117
49996,0.943468,3,5,192,231,139,225
49997,0.760175,4,250,221,168,101,18
49998,0.148035,4,234,248,140,133,234


# Tratando dados

## Descartando dados duplicados

In [16]:
df_benign[df_benign.duplicated()]

,Rel Time,DLC,Arb ID,Byte 0,Byte 1,Byte 2,Byte 3
24,0.000112,1,165,100,0,0,0
53,0.000112,1,165,100,0,0,0
78,0.000112,1,165,100,0,0,0
103,0.000112,1,165,100,0,0,0
105,0.000134,2,194,88,0,0,0
...,...,...,...,...,...,...,...
393877,0.000133,2,194,54,0,0,0
393880,0.000133,2,194,54,0,0,0
393881,0.120688,3,193,0,0,0,0
393882,0.000132,2,194,54,0,0,0


In [17]:
# Descartando duplicadas
initial_len = df_benign.shape[0]
df_benign = df_benign.drop_duplicates()
print(f'Tamanho inicial: {initial_len}, tamanho final {df_benign.shape[0]} | Descartadas {initial_len - df_benign.shape[0]} duplicadas')

Tamanho inicial: 393890, tamanho final 245297 | Descartadas 148593 duplicadas


## DSKDKSJ

In [18]:
def ranking_IDs(df):
    valores_unicos = sorted(df['Arb ID'].unique().tolist())
    
    le = LabelEncoder()

    le.fit(valores_unicos)

    df['Arb ID'] = le.fit_transform(df['Arb ID'])

    return df, le

In [19]:
df_benign, le = ranking_IDs(df_benign)

df_malicious['Arb ID'] = le.fit_transform(df_malicious['Arb ID'])

C:\Users\henri\AppData\Local\Temp\ipykernel_43296\2544822250.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Arb ID'] = le.fit_transform(df['Arb ID'])


In [20]:
df_benign

,Rel Time,DLC,Arb ID,Byte 0,Byte 1,Byte 2,Byte 3
0,0.000000,1,5,100,0,0,0
1,0.001296,1,5,100,0,0,0
2,0.001552,1,5,100,0,0,0
3,0.000112,1,5,100,0,0,0
4,0.000906,1,5,100,0,0,0
...,...,...,...,...,...,...,...
393884,0.106460,3,14,0,0,0,0
393885,0.000184,2,15,54,0,0,0
393886,0.046381,4,13,244,146,0,37
393887,0.076806,3,14,0,0,0,0


In [21]:
df_malicious

,Rel Time,DLC,Arb ID,Byte 0,Byte 1,Byte 2,Byte 3
0,0.205494,3,74,146,15,251,125
1,0.124802,2,222,10,43,197,147
2,0.047389,1,70,79,220,195,180
3,0.361882,3,156,26,113,192,130
4,0.450875,4,129,209,249,55,67
...,...,...,...,...,...,...,...
49995,0.508828,2,193,104,139,17,117
49996,0.943468,3,5,192,231,139,225
49997,0.760175,4,250,221,168,101,18
49998,0.148035,4,234,248,140,133,234


# Análise exploratória

In [10]:
df_benign['DLC']

0         1
1         1
2         1
3         1
4         1
         ..
393884    3
393885    2
393886    4
393887    3
393889    4
Name: DLC, Length: 245297, dtype: int64

# IAs

## OCSVM

In [11]:
model_ocsvm = OneClassSVM(nu=0.1688633942226217, kernel="rbf", gamma='scale', cache_size=10000000000000)
model_ocsvm.fit(df_benign)

OneClassSVM(cache_size=10000000000000, nu=0.1688633942226217)

In [12]:
#dump(model_ocsvm, "models/model_ocsvm.joblib")

['models/model_ocsvm_alternative.joblib']

In [22]:
model_ocsvm.score_samples(df_malicious)

array([1367.33712747,   53.10026966,  286.67182502, ...,  142.13514138,
         19.18314196, 2722.27644944])

In [23]:
ocsvm_predicoes = model_ocsvm.predict(df_malicious)
ocsvm_predicoes

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [25]:
elemento = -1

# Contando as ocorrências do elemento
np.count_nonzero(ocsvm_predicoes == elemento)

49796

In [31]:
elemento = 1

# Contando as ocorrências do elemento
np.count_nonzero(ocsvm_predicoes == elemento)

204